In [30]:
%pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


933.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [31]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import math
import pandas as pd

## Модели для получения эмбеддингов

### DeepPavlov: rubert-base-cased-sentence

In [32]:
def rubert_base_cased_emb(comments): 
    
    rubert_base_cased_name = "DeepPavlov/rubert-base-cased-sentence"
    rubert_base_cased_tokenizer = AutoTokenizer.from_pretrained(rubert_base_cased_name)
    rubert_base_cased_model = AutoModel.from_pretrained(rubert_base_cased_name)
    
    inputs = []
    for comment in comments:
        inputs.append(rubert_base_cased_tokenizer(comment,return_tensors="pt"))

    outputs = []
    for input in inputs:
        outputs.append(rubert_base_cased_model(**input))
        
    return inputs, outputs, rubert_base_cased_tokenizer

### ...

## Близость выходного эмбеддинга с эмбеддингами токенов  

### Метрика: скалярное произведдение

In [33]:
def scalar_product_metric(model_inputs, model_outputs, tokenizer): 
    
    dicts_scalar_product = list(dict())
    
    for iterator in range(len(model_outputs)):

        dicts_scalar_product.append({})

        for tokenId in range(len(model_inputs[iterator]['input_ids'][0])):

            scalar_product = np.dot(model_outputs[iterator][0][0][tokenId].data, model_outputs[iterator][1][0].data)

            dicts_scalar_product[iterator][scalar_product] = model_inputs[iterator]['input_ids'][0][tokenId].data
            
    lists_scalar_product = []
    lists_scalar_product_tokens = []

    for dictId in range(len(dicts_scalar_product)):
    
        lists_scalar_product.append([])
        lists_scalar_product_tokens.append([])
    
        for key in sorted(dicts_scalar_product[dictId])[::-1]:

            lists_scalar_product[dictId].append([key, tokenizer.convert_ids_to_tokens(int(dicts_scalar_product[dictId][key].data))])
            lists_scalar_product_tokens[dictId].append(tokenizer.convert_ids_to_tokens(int(dicts_scalar_product[dictId][key].data)))

    return lists_scalar_product_tokens

### Метрика: косинусное растояние

In [34]:
def сosine_distance_metric(model_inputs, model_outputs, tokenizer): 
    
    dicts_cos = list(dict())

    for iterator in range(len(model_outputs)):

        dicts_cos.append({})
        len_comment_embedding = np.dot(model_outputs[iterator][1][0].data, model_outputs[iterator][1][0].data)**(1/2)

        for tokenId in range(len(model_inputs[iterator]['input_ids'][0])):

            len_token_embedding = np.dot(model_outputs[iterator][0][0][tokenId].data, model_outputs[iterator][0][0][tokenId].data)**(1/2)
            cos = np.dot(model_outputs[iterator][0][0][tokenId].data, model_outputs[iterator][1][0].data)/(len_comment_embedding * len_token_embedding)

            dicts_cos[iterator][cos] = model_inputs[iterator]['input_ids'][0][tokenId].data
            
    lists_cos = []
    lists_cos_tokens = []

    for dictId in range(len(dicts_cos)):
    
        lists_cos.append([])
        lists_cos_tokens.append([])

        for key in sorted(dicts_cos[dictId])[::-1]:

            lists_cos[dictId].append([key, tokenizer.convert_ids_to_tokens(int(dicts_cos[dictId][key].data))])
            lists_cos_tokens[dictId].append(tokenizer.convert_ids_to_tokens(int(dicts_cos[dictId][key].data)))
        
    return lists_cos_tokens

### ...

## Dataset

In [35]:
comments = ["3D Touch просто восхитительная вещь! Заряд держит целый день. В розовом цвете смотрится очень необычно. Touch ID очень быстрый и удобный. Всем советую этот телефон!",
            "Отключается при температуре близкой к нулю, непонятно ведет себя батарея",
            "Достойный телефон. Пользоваться одно удовольствие.",
            "Проблемы чисто софтверные, и выражаются в явном неуважении компании к своим клиентам.",
            "Я для себя понял, что не стоит брать продукцию Apple. Купил эту ошибку природы за 54990 рублей, лучше бы купил Yota Phone.",
            "Телефон больше модный,мажорный чем прогрессивный.Качество сборки на уровне дешевых аппаратов.",
            "я не знаю почему в отзывах ничего не сказано про аудио выход,тк он просто отвратный",
            "Уже несколько часов пользуюсь этим телефоном, до этого был айфон 4, все устраивает.",
            "Брал телефон на тест-драйв (смущал очень сильно размер телефона). Рассматриваю  замену своей 620. Тест-драйвом доволен, телефоном тоже, но не хватает LTE. Буду брать не росттест.",
            "Зря убитое время с ним, буду продавать и покупать андроид в этой же ценовой категории. Советую ни кому не покупать до тех пор пока в маркете приложений не будет достаточных приложений как и в андроиде.",
            "свои деньги он окупает за полгода пользования подвергался падениям с разных высот ,но все работает идеально",
            "Если Вам достаточно 64 Gb, то лучше посмотреть в сторону 5S. По быстродействию практически не отличаются.",
            "телефон вполне устраивает, все работает стабильно, редко подглючивает при включении камеры(картинка зависает) но скидываешь перезаходишь и все в норме, пользуюсь 10 месяцев",
            "Это мой третий айфон по счету и последний. Больно уж много стало косяков. На что переходить не знаю, свои проблемы есть в любой системе, а было все.",
            "В целом, телефон отличный. Других смартфонов по такой цене и характеристикам просто нет. Есть недоработки, но за такую цену их можно простить. По моему мнению, нет смысла переплачивать за бренд. Советую приобрести эту модель."
           ]

## Тесты

In [36]:
def test(comments, model, metric):
    
    model_input, model_output, tokenizer = model(comments)
    lists_tokens = metric(model_input, model_output, tokenizer)
    
    for index in range(len(comments)):

        print(comments[index])
        print(lists_tokens[index])
        print('\n')
    

### Модель: rubert-base-cased-sentence

In [37]:
def rubert_base_cased_sentence_test(comments, metric):
    test(comments=comments, model=rubert_base_cased_emb, metric=metric)

#### Метрика: Скалярное произведение

In [38]:

rubert_base_cased_sentence_test(comments=comments, metric=scalar_product_metric)

3D Touch просто восхитительная вещь! Заряд держит целый день. В розовом цвете смотрится очень необычно. Touch ID очень быстрый и удобный. Всем советую этот телефон!
['Touch', '3D', 'розов', 'Touch', 'телефон', 'ID', 'цвете', '##ом', '[SEP]', '.', '!', '[CLS]', 'Заряд', '.', '!', 'быстрый', '.', 'необычно', 'день', 'вещь', 'В', '##ю', 'смотрит', 'удобный', 'держит', 'восхит', 'целый', 'этот', 'совету', 'Всем', 'очень', '##ся', 'очень', 'и', '##ительная', 'просто']


Отключается при температуре близкой к нулю, непонятно ведет себя батарея
['себя', 'к', 'ведет', '##ается', '[SEP]', '[CLS]', 'нулю', 'близкой', 'при', 'непонятно', '##люч', ',', 'температуре', 'Отк', 'батарея']


Достойный телефон. Пользоваться одно удовольствие.
['Польз', '[SEP]', 'телефон', '[CLS]', '.', '##йн', 'удовольствие', '.', '##ый', '##оваться', 'Досто', 'одно']


Проблемы чисто софтверные, и выражаются в явном неуважении компании к своим клиентам.
['##м', 'в', 'явно', 'к', 'выражаются', 'компании', '[SEP]', 'клиен

#### Метрика: Косинусное растояние

In [39]:
rubert_base_cased_sentence_test(comments=comments, metric=сosine_distance_metric)

3D Touch просто восхитительная вещь! Заряд держит целый день. В розовом цвете смотрится очень необычно. Touch ID очень быстрый и удобный. Всем советую этот телефон!
['Touch', '3D', 'розов', 'Touch', 'телефон', 'ID', 'цвете', '##ом', '[SEP]', '.', '!', '[CLS]', 'Заряд', '.', '!', 'быстрый', '.', 'необычно', 'день', 'вещь', 'В', '##ю', 'смотрит', 'удобный', 'держит', 'восхит', 'целый', 'этот', 'совету', 'Всем', 'очень', '##ся', 'очень', 'и', '##ительная', 'просто']


Отключается при температуре близкой к нулю, непонятно ведет себя батарея
['себя', 'к', 'ведет', '##ается', '[SEP]', '[CLS]', 'нулю', 'близкой', 'при', 'непонятно', '##люч', ',', 'температуре', 'Отк', 'батарея']


Достойный телефон. Пользоваться одно удовольствие.
['Польз', '[SEP]', 'телефон', '[CLS]', '.', '##йн', 'удовольствие', '.', '##ый', '##оваться', 'Досто', 'одно']


Проблемы чисто софтверные, и выражаются в явном неуважении компании к своим клиентам.
['##м', 'в', 'явно', 'к', 'выражаются', 'компании', '[SEP]', 'клиен